## Julia installation

### Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.11.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict("JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [ ]:
versioninfo()

Julia Version 1.11.1
Commit 8f5b7ca12ad (2024-10-16 10:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

In [1]:
struct ISymb
    args::Tuple{Vararg{Int64}}
    weight::Int64
end

In [4]:
println(ISymb((0,1,1,1,2), 2))

ISymb((0, 1, 1, 1, 2), 2)


## $\mathbb I^{\text{Symb}}$

$$
I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{n_m-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})
$$

for example, in depth $2$, $I(a_0; 0^{1-1}, a_1, 0^{3-1}, a_2, 0^{2-1}; a_3)\leftrightsquigarrow(0,1,1,3,2,2,3)$

The total ordering is $(i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})\prec(i'_0, n'_0, i'_1, n'_1, \cdots, i'_{m'}, n'_{m'}, i'_{m'+1})$
- if: $\sum n_k < \sum n'_k$
- else if: $i_{m+1} < i'_{m'+1}$
- else: $n_{m'-r} > n'_{m'-r}$ or $i_{m-r} - i_{m-r-1} > i'_{m'-r} - i'_{m'-r-1}$ and the entries to the right coincide

### $\Phi$ map

- $\Phi(I(a; b)) = 1,\quad \Phi(I(0; \cdots; 0)) = 0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};0))=(-1)^{n_0+\cdots+n_m-1}\Phi(I(0;0^{n_m-1},a_{i_m},\cdots,0^{n_0-1};a_{i_0})),\quad a_{i_0}\neq0$

- $\Phi(I(a_{i_0};0^{n_0-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))\\
\displaystyle=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(I(a_{i_0};\cdots,a_{i_k},0^{p-1};0))\Phi(I(0,0^{q-1},a_{i_{k+1}},\cdots;a_{i_{m+1}})),\quad a_{i_0}, a_{i_{m+1}} \neq 0$

- $\Phi(I(0;0^{n_0-1},a_{i_1},0^{n_1-1},\cdots,a_{i_m},0^{n_m-1};a_{i_{m+1}}))=\\
\displaystyle\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{\log^{p_0}(a_{i_{m+1}})}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\left[\frac{a_{i_2}}{a_{i_1}},\cdots,\frac{a_{i_{m+1}}}{a_{i_m}}\right]_{n_1+p_1,\cdots,n_m+p_m}$

Note: $\log^0(a_{i_{m+1}})$ is taken to be $1$ for $p_0=0$ even if $a_{i_{m+1}}=1$

### Implementation of $\Phi$

- $\Phi(i_0,1,i_1) = 1,\quad \Phi(0,n_0,\cdots,n_m,0) = 0$

- $\Phi(i_0, n_0, \cdots, i_m, n_m, 0)=(-1)^{n_0+\cdots+n_m-1}\Phi(0,n_m,i_m,\cdots,n_0,i_0),\quad i_0\neq0$

- $\displaystyle\Phi(i_0,n_0,\cdots,i_m,n_m,i_{m+1})=\sum_{k=0}^m\sum_{p+q=n_k>1}\Phi(i_0,\cdots,i_k,p;0)\Phi(0,q,i_{k+1},\cdots;i_{m+1}),\quad i_0,i_{m+1} \neq 0$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,d+1)=\sum_{p_1+\cdots+p_m=n_0-1}(-1)^{n_0+m-1}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,d+1-i_m)$

- $\displaystyle\Phi(0;n_0,i_1,n_1,\cdots,i_m,n_m,i_{m+1})=\sum_{p_0+\cdots+p_m=n_0-1}(-1)^{n_0+p_0+m-1}\dfrac{\log^{p_0}(a_{i_{m+1}})}{p_0!}\binom{n_1+p_1-1}{n_1-1}\cdots\binom{n_m+p_m-1}{n_m-1}\\(i_1,n_1+p_1,i_2-i_1,\cdots,i_m-i_{m-1},n_m+p_m,i_{m+1}-i_m)$

## $\mathbb H^{\text{Symb}}$

$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow(i_1,n_1,i_2-i_1,\cdots,i_d-i_{d-1},n_d,i_{d+1}-i_d)
$$

Or $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ so that $i_r = m_1 + \cdots + m_r$

The total ordering is $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\prec(m'_1,n'_1,\cdots,m'_{d'},n'_{d'},m'_{d'+1})$
- if: $\sum n_k < \sum n'_k$
- else if: $\sum m_k < \sum m'_k$
- else: $m_{d-r} > m'_{d'-r}$ or $n_{d-r} > n'_{d'-r}$ and the entries to the right coincide

The $\partial_{r}$ of $(m_1,n_1,\cdots,m_d,n_d,m_{d+1})$ is
- if $d == 1$ and $n_1 == 1$: $-dv_{i_1,i_2-1}$
- else if $n_r>1$ or $d=1$: $(\cdots,n_r-1,\cdots) du_{i_r, i_{r+1}-1}$
- else if $r=d$: $-(\cdots,n_{d-1},m_d+1+m_{d+1}) dv_{i_d,i_{d+1}-1}$
- else: $-(\cdots,m_{r}+1+m_{r+1},n_{r+1},\cdots) dv_{i_r, i_{r+1}-1} + (\cdots,m_r,n_{r+1},m_{r+1}+1+m_{r+2},\cdots) (du_{i_r, i_{r+1}-1} - dv_{i_r, i_{r+1}-1})$

Since
$$
[x_{i_1\to i_2},\cdots,x_{i_d\to i_{d+1}}]_{n_1,\cdots,n_d}\leftrightsquigarrow I(a_0;a_{i_1},0^{n_1-1},\cdots,a_{i_d},0^{n_d-1};a_{i_{d+1}})
$$
so `toISymb` is implemented as
$$
(m_1,n_1,\cdots,m_d,n_d,m_{d+1})\rightsquigarrow(0,1,i_1,n_1,\cdots,i_d,n_d,i_{d+1})
$$

## $\mathbb L^{\text{Symb}}$

## $T(\mathbb H^{\text{Symb}})$

$$
\left(\sum_{i_1}c_{i_i}^{(1)}t_{i_1}^{(1)}\right)\otimes\cdots\otimes\left(\sum_{i_N}c_{i_N}^{(N)}t_{i_N}^{(N)}\right)=\sum_{i_1,\cdots,i_N}c_{i_i}^{(1)}\cdots c_{i_N}^{(N)} t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(N)}
$$

$$
\left(\sum_{\mathbf i}c_{\mathbf i}^{(1)}t_{i_1}^{(1)}\otimes\cdots\otimes t_{i_N}^{(1)}\right)\left(\sum_{\mathbf j}c_{\mathbf j}^{(2)}t_{j_1}^{(2)}\otimes\cdots\otimes t_{j_N}^{(2)}\right)=\sum_{\mathbf i,\mathbf j}c_{\mathbf i}^{(1)}c_{\mathbf j}^{(2)}\left(t_{i_1}^{(1)}t_{j_1}^{(2)}\right)\otimes\cdots\otimes\left(t_{i_N}^{(1)}t_{j_N}^{(2)}\right)
$$

## $\bigwedge^*\mathbb L^{\text{Symb}}$

## Variation matrix over $S_d(\mathbb C)$

Recall the *complementary entry* of $(-1)^kI(0;a_{i_1},0^{m_{i_1}-1},\cdots,a_{i_k},0^{m_{i_k}-1};1)$ with respect to $(-1)^lI(0;a_{j_1},0^{p_{j_1}-1},\cdots,a_{j_l},0^{p_{j_l}-1};1)$ is
$$
(-1)^{l-k} I^{\sigma_{i_1}\sigma_0^{m_{i_1}-1}\cdots\sigma_{i_k}\sigma_0^{m_{i_k}-1}}(0;a_{j_1},0^{p_{j_1}-1},\cdots,a_{j_l},0^{p_{j_l}-1};1)
$$

$\sigma_{i_1}\sigma_0^{m_1-1}\cdots\sigma_{i_k}\sigma_0^{m_k-1}$ should correspond to $(0,1,i_1,m_1,\cdots, i_k,m_k)$

Suppose $\{i_1,\cdots,i_k\}=\{j_{q_1},\cdots,j_{q_k}\}$ is a subsequence of $\{j_1,\cdots,j_l\}$ (otherwise the complementary entry is 0), then the complementary entry is

$$
(-1)^{l-k}I(0;\cdots;a_{j_{q_1}})\left(\prod_{r=1}^{k-1} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})\right)I^{\sigma_0^{m_k-1}}(a_{j_{q_k}};\cdots;1)
$$

Assume
- $a_0=0$, $a_{d+1}=1$
- $i_0=j_0=0$, $i_{k+1}=j_{l+1}=d+1$ so that $q_0=0$, $q_{k+1}=l$,
- $m_0=1$

then the complementary entry is
$$
(-1)^{l-k}\prod_{r=0}^{k} I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})
$$
If $m_r > 1$, it is equal to
$$
I^{\sigma_0^{m_r-1}}(a_{j_{q_r}};\cdots;a_{j_{q_{r+1}}})=\sum_{\substack{q_r\leq s<q_{r+1}-1 \\ m_r \leq p_s}}\sum_{u+v=p_s-m_r}I(a_{j_{q_r}};\cdots,a_{j_s},0^{u};0)I(0;0^{v},a_{j_{s+1}},\cdots;a_{j_{q_{r+1}}})
$$



`complementaryEntry(w1, w2)`: the complementary entry of $w_1 = (0,1,j_1,p_1,\cdots,j_l,p_l)$ and $w_2 = (0,1,i_1,m_1,\cdots,i_k,m_k)$. Which should be the product of `Isigma(r)` from $r=0$ to $k$

## One form

## Connection form